### Import Module

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from openpyxl import load_workbook
import time
from datetime import date
import pandas as pd

### Setting driver with Path and website url

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

chrome_options = Options()
prefs = {'profile.default_content_setting_values': {'images': 2}}

chrome_options.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),chrome_options=chrome_options)

url = "https://www.noon.com/egypt-en/sports-and-outdoors/exercise-and-fitness/yoga-16328/"

driver.get(url)

driver.maximize_window()

# chrome_options = webdriver.ChromeOptions()

C:\Users\Shamshad Choudhary\AppData\Local\Temp\ipykernel_3008\812615092.py:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),chrome_options=chrome_options)


#### below code was written to make excel file to difined no. of columns
#### Therefore on initial run excel file was made after it was commented out on other run to append data in it 

In [3]:
df_bike = pd.DataFrame(columns=['Date','EAN/SKU','Position','Product Name','Currency','Product Price','Old Price','Product Rating','Product Rating Count',
                                   'Brand','Store Name','Store Rating','Store Rating count','FBN','link'])

df_bike.to_excel(f'Extraction.xlsx',index=False)

#### was use to open already made excel file

In [4]:
workbook_name = 'Extraction.xlsx'
wb = load_workbook(workbook_name)
page = wb.active

In [5]:
dates = []
EANs = []
product_names = []
currency = []
new_prices = []
old_prices = []
proRatings = []
proRatingCounts = []
brands = []
storeNames = []
positions = []
storeRatingCounts = []
storeRatings = []
FBNs = []

#### crashing of program due to internet issues.Therefore, i am using for loop upto range of 5 to make 5 pages excel at time
#### making 5 pages product details excel at time and merged all .xlsx files to create single 3000 porduct details excel file manually

In [ ]:
for j in range(0,1):              
    for i in range(j*5,(j+1)*5):   ## scarping 5 pages at time due to internet problem which causes program to crash
        links = []
        print("Scraping page ",i+1)
        products = driver.find_elements(By.XPATH,"(//div[@class='sc-cb3f65f3-7 pEiif grid']/span/a)")

        for link in products:
            links.append(link.get_attribute('href'))

            EAN = link.get_attribute('id').split('-')
            EANs.append(EAN[1])
        
        count = 0
        

        for link in links:
            driver.get(link)

            data = []
            
          # Locators

            today = date.today()

            try:
                position = driver.find_element(By.XPATH,"//div[@class='sc-e4a456cc-5 hsjerl']").text
            except:
                position = ""

            try:
                name = driver.find_element(By.XPATH,"//h1[@class='sc-c44e3e2d-12 kODUYj']").text
            except:
                name = ""

            try:
                newPrice = driver.find_element(By.XPATH,"//div[@class='priceNow']").text
            except:
                newPrice = 0

            try:
                oldPrice = driver.find_element(By.XPATH,"//div[@class='priceWas']").text
            except:
                oldPrice = ""

            try:
                proRating = driver.find_element(By.XPATH,"//span[@class='sc-31ebbe94-1 bngoaw']").text
            except:
                proRating = ""
        
            try:
                proRatingCount = driver.find_element(By.XPATH,"//div[@class='sc-53a159dc-0 iqpeOg']/span").text   
            except:
                proRatingCount = ""

            try:
                brand = driver.find_element(By.XPATH,"//div[@class='sc-c44e3e2d-11 jbKJDi']").text
            except:
                brand = ""

            try:
                storeName = driver.find_element(By.XPATH,"//span[@class='allOffers']").text
            except:
                storeName = ""

            try:
                fbn = driver.find_element(By.XPATH,"//img[@class='sc-b51db3f-1 bGljQY']").get_attribute('alt')
                fbn = fbn.split('-')
                fbn = fbn[1]
            except:
                fbn = 0
            
          
            
            position = position.split('\n')    #making position
            position = position[0:4]
            position = "/".join(position)


            # storeLink = driver.find_element(By.XPATH,"//button[@class='sc-d711b2ac-6 fssgSV']")
            try:
                storeLink = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH,"//button[@class='sc-d711b2ac-6 fssgSV']"))
                )
                storeLink.click()
            except:
                print(f'storeLink not found of {i} page product link: {link}')
        
            try:
                storeRating = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH,"//div[@class='sc-33d82e78-1 ekJjKz']"))
                )
                storeRating = storeRating.text
            except:
                storeRating = ""
        
            try:
                storeRatingCount = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH,"(//div[@class='sc-33d82e78-0 hbyggc']/div)[3]"))
                )
                storeRatingCount = storeRatingCount.text
            except:
                storeRatingCount = ""
        
        ### append data to respective lists
            dates.append(today)
            positions.append(position)
            product_names.append(name)
            currency.append(newPrice.split()[0] if newPrice else newPrice)
            new_prices.append(newPrice.split()[1] if newPrice else newPrice)
            old_prices.append(oldPrice.split()[1] if oldPrice else oldPrice)
            proRatings.append(proRating)
            proRatingCounts.append(proRatingCount.split()[0] if proRatingCount else proRatingCount)
            brands.append(brand)
            storeNames.append(storeName)
            FBNs.append(fbn)
            storeRatings.append(storeRating)
            storeRatingCounts.append(storeRatingCount.split()[2] if storeRatingCount else storeRatingCount)                                     
            
            ## making list to append data in excel (should be proper format according to headers)
            data.append(today)
            data.append(EANs[count])
            data.append(position)
            data.append(name)
            data.append(newPrice.split()[0] if newPrice else newPrice)
            data.append(newPrice.split()[1] if newPrice else newPrice)
            data.append(oldPrice.split()[1] if oldPrice else oldPrice)
            data.append(proRating)
            data.append(proRatingCount.split()[0] if proRatingCount else proRatingCount)
            data.append(brand)
            data.append(storeName)
            data.append(storeRating)
            data.append(storeRatingCount.split()[2] if storeRatingCount else storeRatingCount) 
            data.append(fbn)
            data.append(link)
            
            ## append data in excel file
            page.append(data)
            wb.save(filename=workbook_name)
            
            count=count+1
            
            ## Going back ward
            time.sleep(0.5)
            driver.back()
            time.sleep(0.5)
            driver.back()

        
#         next_page = driver.find_element(By.XPATH,"(//div[@class='sc-31bd7555-1 cpvGCn'])[2]")
#         next_page.click()
        # navigate to next page
        try:
            next_page = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH,"(//div[@class='sc-31bd7555-1 cpvGCn'])[2]"))
            )
            next_page.click()
        except:
            print(f'next_page not found of {i+1} page not clickable')
        

#     df_bike = pd.DataFrame(columns=['Date','EAN/SKU','Position','Product Name','Currency','Product Price','Old Price','Product Rating','Product Rating Count',
#                                    'Brand','Store Name','Store Rating','Store Rating count','FBN','link'])

#     for i in range(len(links)):
#         df_bike = df_bike.append({'Date':dates[i],'EAN/SKU':EANs[i],'Position':positions[i],'Product Name': product_names[i],'Currency':currency[i],'Product Price': new_prices[i],
#                                  'Old Price':old_prices[i],'Product Rating':proRatings[i],'Product Rating Count':proRatingCounts[i],
#                                  'Brand':brands[i],'Store Name':storeNames[i],'Store Rating':storeRatings[i],
#                                  'Store Rating count':storeRatingCounts[i],'FBN':FBNs[i],'link':links[i]}, ignore_index=True)

#     df_bike.to_excel(f'Output{i+1}.xlsx',index=False)

Scraping page  1


In [ ]:
# driver.quit()

In [ ]:
# df_bike = pd.DataFrame(columns=['Date','EAN/SKU','Position','Product Name','Currency','Product Price','Old Price','Product Rating','Product Rating Count',
#                                'Brand','Store Name','Store Rating','Store Rating count','FBN','link'])

In [ ]:
# df_bike

In [ ]:
# for i in range(len(product_names)):
#     df_bike = df_bike.append({'Date':dates[i],'EAN/SKU':EANs[i],'Position':positions[i],'Product Name': product_names[i],'Currency':currency[i],'Product Price': new_prices[i],
#                              'Old Price':old_prices[i],'Product Rating':proRatings[i],'Product Rating Count':proRatingCounts[i],
#                              'Brand':brands[i],'Store Name':storeNames[i],'Store Rating':storeRatings[i],
#                              'Store Rating count':storeRatingCounts[i],'FBN':FBNs[i],'link':links[i]}, ignore_index=True)

In [ ]:
# df_bike

In [ ]:
# df_bike.to_excel('Output.xlsx',index=False)

In [ ]:
# driver.quit()